원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

## 5.5 QR 분해

만일 모든 매개변수가 선형적으로 나타나고 관측값의 수가 기저함수의 수 보다 많다면, 우리는 최소자승 문제를 가지고 있는 것이 된다. 설계행렬 $X$ 는 $m \times n$ 이고 $m>n$ 이다. 우리가 풀고 싶은 문제는 다음과 같다.

$$X \beta \approx y$$

그러나 이 연립방정식은 과결정이다. 방정식이 미지수보다 많다. 따라서 우리는 이 연립방정식의 엄밀하게 풀 수 없다. 사실, 우리는 최소자승 관점에서 푼다:
$$ min_\beta ||X \beta - y ||$$
과결정된 연립방정식을 푸는 한가지 가능한 이론적 접근 방법으로 양변을 $X^T$ 로 곱하는 방법이 있다. 이렇게 하면 해당 연립방정식이 $n \times n$ 의 *정규 방정식* 이라고 알려진 형태로 줄어든다. 

$$X^T X \beta = X^T y$$

관찰값의 수는 수천이지만 미지수의 수는 몇개 밖에 안될 경우, 설계행렬 $X$는 상당히 커지지만, 해당 행렬 $X^TX$는 작아진다. $y$ 는 $X$의 각 열로 생성된 공간에 투사된다. 이러한 이론적 접근을 이어가서, 만일 기저함수가 독립이라면, $X^T X$는 비특이 행렬이 되고 매개변수는 다음과 같이 구해진다.

$$\beta = (X^T X)^{-1} X^T y $$

선형 최소 자승 문제를 푸는 이 공식은 거의 모든 통계와 수치 해석 교과서에 나타난다. 그러나, 몇가지 바람직하지 않은 면이 이 이론적 접근 방법에 있다. 우리는 이미 역행렬을 사용해서 연립방정식을 푸는 것이 가우스 소거법으로 푸는 것 보다 더 수고로우면서 덜 정확하다는 것을 보았다. 그러나, 더 중요한 것은, 정규 방정식은 그 원래 과결정 연립방정식 보다 특이행렬에 더 가깝다는 것이다. 사실, 행렬의 조건수가 제곱이 된다:

$$\kappa (X^T X) = \kappa(X)^2$$

유한 정밀도 연산에서는, 실제로 $X$의 해당 열이 독립적이 된다고 하더라도 정규 방정식의 $(X^TX)$은 특이행렬이 되어 $(X^TX)^{-1}$ 이 존재하지 않게 될 수 있다.

한 극단적인 예로 다음과 같은 설계 행렬을 고려해 보라.

\begin{equation}
X=
  \begin{bmatrix}
    1 & 1 \\
    \delta & 0 \\
    0 & \delta
  \end{bmatrix}
\end{equation}

만일 $\delta$ 가 작다면, $X$의 두 열은 거의 평행하겠지만 선형 독립일 것이다. 해당 정규 방정식에서는 상황이 더 나쁘게 된다:

\begin{equation}
X^T X =
  \begin{bmatrix}
    1+ \delta^2 & 1 \\
    1 & 1 +\delta^2
  \end{bmatrix}
\end{equation}

만일 $|\delta| < 10^{-8}$ 인 경우, 행렬 $(X^TX)$를 배정도 부동소숫점으로 연산하면 정확히 특이행렬이 되고 고전적인 교과서 공식 안에서 요구되는 역행렬은 존재하지 않는다.

QR 분해라고 알려져 있는 *직교화* 알고리듬을 주로 사용하여 정규 방정식을 피하는 방법이 있다. 


In [ ]:
import numpy as np
import numpy.linalg as na


delta = 1e-8
X = np.matrix([[1, 1],
              [delta, 0],
              [0, delta]])

y = np.matrix([[0, 1, 2]]).T

위에서 ``delta`` 값이 작으면 ``X`` 행렬의 두 열은 거의 선형 종속이 된다. QR 분해는 아래 조건을 만족하는 두 행렬 Q와 R을 찾는 것이다.

\begin{align}
X \beta &= y \\
QR &= X \\
Q^TQ &= I
\end{align}

따라서 원래 방정식의 $X$를 $QR$로 바꾸면 다음과 같다
$$ QR \beta = y $$

$Q^T Q = I$ 임을 이용하기 위해 양변 왼쪽에 $Q^T$ 를 곱하면 
$$ Q^TQR \beta = Q^Ty $$

단위 행렬 $I$는 곱해도 결과에 영향을 끼치지 않으므로
$$ R \beta = Q^Ty $$

라고 표시할 수 있다 $z=Q^Ty$ 라고 하면
$$ R \beta = z $$

$R$은 정방행렬이므로 $\beta$를 다음과 같이 구할 수 있을 것이다.
$$\beta = R^{-1} z$$

이러한 절차를 아래와 같이 구할 수 있다. 우선 $z=Q^Ty$ 를 구한다.

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html
Q, R = na.qr(X)
print('Q = %s' % Q)
print('R = %s' % R)
z = Q.T * y
print('z = %s' % z)

$\beta$ 를 구하기 위해 다음 방정식을 푼다.
$$ R \beta = z $$

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html
beta = na.solve(R, z)
print('beta = %s' % beta)

잔차는 다음과 같이 구할 수 있다. 우리의 목적은 이것을 0으로 만드는 것이라기 보다는 최소로 줄이는 것이다.
$$err = X \beta -y$$

잔차가 크거나 작다고 말하고자 한다면, 놂을 계산하여야 한다.
$$||err||^2 = \sqrt{err^Terr}$$


In [ ]:
err = X * beta - y
print('err = %s' % err)
print('||err||2 = %s' % np.sqrt(err.T * err))

한편 위에서 설명했던 이론적인 접근은 다음과 같은 결과를 보여 준다. (여러 가지 $\delta$ 값을 시도해 보라)

In [ ]:
try:
    beta_left_inv = (X.T * X).I*X.T * y
except na.LinAlgError: 
    print('(X.T * X).shape = %s' % str((X.T * X).shape))
    print('rank(X.T * X) = %r' % na.matrix_rank(X.T * X))
else:
    err_left_inv = X * beta_left_inv - y
    print('err_left_inv = %s' % err_left_inv)
    print('||err_left_inv||2 = %s' % np.sqrt(err_left_inv.T*err_left_inv))

$m \times n$ 인 $X_{m \times n}$ 로 부터 $Q$, $R$ 을 정하는 방법은 두가지가 있다.
\begin{align}
X_{m \times n} &= Q_{m \times m}R_{m \times n} \\
X_{m \times n} &= Q_{m \times n}R_{n \times n}
\end{align}

$Q$ 은 *직교 Orthogonal* 의 $O$ 대신이고 $R$ 은 *우삼각 Right triangle*의 $R$이다. 많은 선형 대수 교과서에 나오는 그람 슈미트 직교화도 같은 결과를 얻을 수 있으나 수치적으로는 덜 만족스럽다.

$R$행렬은 일련의 하우스홀더 거울상을 $X$의 각 열에 반복하여 만든다:
$$H_n  \cdots H_2 H_1 X = R$$

$R$의 $j$ 번째 열은 $X$의 첫 $j$ 열의 선형 조합이다. 따라서, $R$의 주 대각선 아래 요소는 0이다.

만일 같은 일련의 하우스홀더 거울상을 우변에도 적용한다면, 방정식
$$X \beta \approx y$$
는 
$$R \beta \approx z$$
가 되는데 여기서
$$H_n  \cdots H_2 H_1 y = z$$
이다.

이 연립 방정식의 첫 $n$ 개의 방정식은 작고, 행과 열의 크기가 같은 삼각 연립방정식으로 후진대입법으로 $\beta$를 풀 수 있다. 남은 $m-n$ 개의 방정식의 계수는 모두 0이므로, 이러한 방정식들은 $\beta$ 에 독립적이고 따라서 여기에 속한 $z$의 요소가 변환된 잔차를 이룬다. 이런 접근 방식이 정규 방정식에 비해 더 선호되는데, 그 까닭은 하우스홀더 거울상이 나무랄데 없는 수치적 신뢰도를 가지고 있고, 결과로 얻어지는 삼각 연립 방정식이 후진대입법을 적용할 준비가 되어 있기 때문이다.

$QR$ 분해의 $Q$ 행렬은 다음과 같다.
$$Q=(H_n  \cdots H_2 H_1)^T$$

최소자승문제를 풀기 위해 실제로 $Q$를 계산할 필요는 없다. 해당 분해법의 다른 용도를 위해서는 $Q$를 명시적으로 가지고 있는 것이 편리할 수도 있다. 만일 우리가 첫 $n$ 열만 계산한다면 우리는 축약된 분해법의 결과($Q_{m \times n}R_{n \times n}$)를 가지게 된다. $m$ 열을 모두 계산한다면 전체 결과($Q_{m \times m}R_{m \times n}$)를 가지게 된다. 어쨌든 
$$Q^T Q = I$$
로 $Q$의 각 열은 서로 직교하고 모두 크기가 1이다. 그러한 행렬은 *열*이 *단위직교*이다라고 말한다. 전체 $Q$의 경우, 
$$QQ^T = I$$
이므로 전 $Q$ 는 *단위직교* 행렬이다.

이것을 어떤 작은 인구 조사 예제로 표시해 보도록 하자. 우리는 1950년대 부터 2000년대 까지의 여섯 측정값을 2차 곡선으로 적합할 것이다.
$$y(s) \approx \beta_1 s^2 + \beta_2 s + \beta_3$$
변환된 시간 $s$와 측정값 $y$ 는 다음과 같다.


In [ ]:
import numpy as np

s = (np.matrix([np.arange(1950, 2000+1, 10)]).T - 1950.0)/50
y = np.matrix([np.array([150.6970, 179.3230, 203.2120, 226.5050, 249.6330, 281.4220])]).T
print('     s                   y')
print(np.column_stack([s, y]))

설계행렬 $X$는

\begin{equation}
X=
  \begin{bmatrix}
        s^2 & s & 1
  \end{bmatrix}
\end{equation}
로 다음과 같이 계산할 수 있다.

In [ ]:
X = np.column_stack([np.power(s,2), s, np.ones_like(s)])
X

첫 단계는 $X$의 첫 열에서 대각 아래 요소를 0으로 만든다.

In [ ]:
def householder_k(x, k):
    """
    Householder reflection such that Hx points to kth axis
    
    :param numpy.matrix x: 
    :param int k: 
    :return: 
    """
    # find u bisecting x and x axis
    sigma = np.sqrt((x.T * x)[0, 0])
    e_k = np.matrix(np.zeros_like(x))
    e_k[k, 0] = 1.0
    u = x + sigma * e_k

    # Householder reflection
    rho = (2 / (u.T * u))[0, 0]
    tau_x = (rho * u.T * x)[0, 0]
    Hx = x - tau_x * u

    return Hx, rho, u


def householder_xy(x, y, k):
    """
    Householder reflection such that Hx points to kth axis
    
    :param numpy.matrix x: 
    :param numpy.matrix y: 
    :param int k: 
    :return: 
    """
    Hx, rho, u = householder_k(x, k)

    tau_y = (rho * u.T * y)[0, 0]
    Hy = y - tau_y * u

    return Hx, Hy, u


x = X[:, 0]

Hx, Hy, u = householder_xy(x, y, 0)
X[:, 0]=Hx
y = Hy
print('X = %s' % X)
print('Hy = %s' % Hy)
